# Example 5. Uncalibrated modulators: Oblique plane structured illumination microscopy

Load raw [oblique plane structured illumination microscopy (OPSIM)](https://www.biorxiv.org/content/10.1101/2022.05.19.492671v1.full), process data for rotation and registration, determine SIM pattern infromation, and run reconstruction.

Instrument designed and built by Bingying Chen, Bo-Jui Chang, and Reto Fiolka.  
Cardiomyocyte cell samples labeled for alpha-actinin 2 by James Hayes and Dylan Burnette.   
All imaging by Reto Fiolka. 

### Download data from Zenodo and extract

Data available from [OPSIM Zenodo repository](https://zenodo.org/record/6481084#.YmVM-7lOmHs).  Please download and extract "1_CH00_000000.tif" into data/example_005/raw_data. Once this is done, you can run this example.

### Import libraries

In [1]:
import numpy as np
from numpy import fft
from pathlib import Path
import mcsim.analysis.sim_reconstruction as sim
from mcsim.analysis import analysis_tools
from localize_psf import rois
import matplotlib.pyplot as plt
from matplotlib.colors import PowerNorm
from skimage.transform import rotate
import napari
import tifffile
import gc
from example005_resources.deskew_opm_data import deskew
import itk

### Define data paths to raw OPSIM stack and 2D PSF extracted from many beads

In [2]:
opsim_file_path = Path("data", "example_005", "raw_data", "1_CH00_000000.tif")
psf_file_path = Path("example005_resources", "psf2d.tif")

### Define experimental metadata

In [3]:
# extracted from text file provided with acquisition
n_colors = 1
n_angles = 3
n_phases = 3
ns = 220
ny = 256
nx = 896
pixel_size = 0.114 # um
ds = 0.320 # um - scan step size, not coverslip z
na = 1.0 # Snouty NA
excitation_wavelengths = 0.488 # um
emission_wavelengths = 0.580 # um
tilt_angle = 45.0

# transformed pixel spacing. qi2lab deskew deskews onto an isotropic grid in xy
coverslip_dz = pixel_size * np.sin(tilt_angle)
coverslip_dy = pixel_size
coverslip_dx = pixel_size

### Load and parse data into angles and phases

In [4]:
# create array to hold all raw opsim images
sim_images = np.zeros((n_colors,n_angles,n_phases,ns,ny,nx),dtype=np.uint16)

# loop through TIFF file and load each raw opsim image into proper metadata location
page_idx = 0
with tifffile.TiffFile(opsim_file_path) as tif:
    n_pages = len(tif.pages)
    for channel_idx in range(n_colors):
        for angle_idx in range(n_angles):
            for phase_idx in range(n_phases):
                for scan_idx in range(ns):
                    sim_images[channel_idx,angle_idx,phase_idx,scan_idx,:,:] = tif.pages[page_idx].asarray()
                    page_idx = page_idx + 1

### Visualize OPM SIM data in native reference frame

In [ ]:
# add images to napari viewer with scale information
viewer = napari.view_image(sim_images,name='Raw OPSIM data',scale=(1,1,1,ds,pixel_size,pixel_size))

# activate scale bar in physical units
viewer.scale_bar.unit = 'um'
viewer.scale_bar.visible = True

# label sliders in napari viewer
viewer.dims.axis_labels = ['c','a','p','s','y','x']

### Calculate deskewed widefield equivalent images for registration
  
Orthogonal deskew function for OPM extracted from [qi2lab OPM package](https://www.github.com/qi2lab/opm). 

In [ ]:
# deskew one channel/angle/phase to get the size of the image
test_deskew = deskew(sim_images[0,0,0,:],tilt_angle,ds,pixel_size)
n_coverslip_z = test_deskew.shape[0]
n_coverslip_y = test_deskew.shape[1]
n_coverslip_x = test_deskew.shape[2]
coverslip_wf_images = np.zeros((n_colors,n_angles,n_coverslip_z,n_coverslip_y,n_coverslip_x),dtype=np.uint16)

# perform deskewing for all widefield equivalent views (average over phases)
for channel_idx in range(n_colors):
    for angle_idx in range(n_angles):
        for phase_idx in range(n_phases):
            image_plane = deskew(np.flipud(np.nanmean(sim_images[channel_idx,angle_idx,:,:,:,:],axis=0)),tilt_angle,ds,pixel_size)
            coverslip_wf_images[channel_idx,angle_idx,:] = image_plane

# clean up variables
del test_deskew
gc.collect()

### Rotate deskewed widefield equivalent images with prior info from instrument

In [ ]:
# pad images to be square
max_size = np.maximum(n_coverslip_y,n_coverslip_x)

if max_size > n_coverslip_y:
    pad_amount_y = np.abs(max_size-n_coverslip_y)//2
else:
    pad_amount_y = 0
if max_size > n_coverslip_x:
    pad_amount_x = np.abs(max_size-n_coverslip_x)//2
else:
    pad_amount_x = 0

pad_width = ((0,0),(0,0),(0,0),(pad_amount_y,pad_amount_y+1),(pad_amount_x,pad_amount_x))

coverslip_rotated_wf_images = np.pad(coverslip_wf_images,pad_width=pad_width)

# rotation angle guesses from instrument design
rotation = [60,0,-60]

# perform rotation across all z planes for each channel + angle
for channel_idx in range(n_colors):
    for angle_idx in range(n_angles):
        for z_idx in range(n_coverslip_z):
            rotated_plane = rotate(coverslip_rotated_wf_images[channel_idx,angle_idx,z_idx,:,:],
                                    angle=rotation[angle_idx],
                                    center=None,
                                    resize=False,
                                    mode='constant',
                                    cval=0,
                                    preserve_range=True)
            coverslip_rotated_wf_images[channel_idx,angle_idx,z_idx,:,:] = rotated_plane

# clean up variables
del coverslip_wf_images
gc.collect()

### Determine image registration in coverslip reference frame using deskewed and rotated widefield images

In [ ]:
# create ITK array for fixed view
# itk-elastix currently supports np.float32 only
channel_idx = 0
angle_idx = 1
fixed_image_a001 = itk.GetImageFromArray(coverslip_rotated_wf_images[channel_idx,angle_idx,:].astype(np.float32))

# create empty rigid transformation object
parameter_object_rigid = itk.ParameterObject.New()
default_rigid_parameter_map = parameter_object_rigid.GetDefaultParameterMap('rigid')
parameter_object_rigid.AddParameterMap(default_rigid_parameter_map)

# create empty affine transformation object
parameter_object_affine = itk.ParameterObject.New()
default_affine_parameter_map = parameter_object_affine.GetDefaultParameterMap('affine',4)
default_affine_parameter_map['FinalBSplineInterpolationOrder'] = ['0']
parameter_object_affine.AddParameterMap(default_affine_parameter_map)

# Call registration functions to determine registration of rotated view (angle_idx=[0,2]) to center view (angle_idx=1)

# create ITK array for +60 rotated view
# itk-elastix currently supports np.float32 only
angle_idx = 0
moving_image_a000 = itk.GetImageFromArray(coverslip_rotated_wf_images[channel_idx,angle_idx,:].astype(np.float32))

# register angle_idx=0 to angle_idx=1
# begin with rigid registration
result_image_a000_rigid, result_transform_parameters_a000_to_a001_rigid = itk.elastix_registration_method(
    fixed_image_a001, moving_image_a000,
    parameter_object=parameter_object_rigid,
    log_to_console=False)

# refine rigid transformation with affine registration
result_image_a000_affine, result_transform_parameters_a000_to_a001_affine = itk.elastix_registration_method(
    fixed_image_a001, result_image_a000_rigid,
    parameter_object=parameter_object_affine,
    log_to_console=False)

# clean up variables for +60 rotated view
del moving_image_a000, result_image_a000_rigid, result_image_a000_affine
gc.collect()

# create ITK array for -60 rotated view
# itk-elastix currently supports np.float32 only
angle_idx = 2
moving_image_a002 = itk.GetImageFromArray(coverslip_rotated_wf_images[channel_idx,angle_idx,:].astype(np.float32))

# register angle_idx=2 to angle_idx=1
# begin with rigid registration
result_image_a002_rigid, result_transform_parameters_a002_to_a001_rigid = itk.elastix_registration_method(
    fixed_image_a001, moving_image_a002,
    parameter_object=parameter_object_rigid,
    log_to_console=False)

# refine rigid transformation with affine registration
result_image_a002_affine, result_transform_parameters_a002_to_a001_affine = itk.elastix_registration_method(
    fixed_image_a001, result_image_a002_rigid,
    parameter_object=parameter_object_affine,
    log_to_console=False)

# clean up variables for -60 rotated view
del fixed_image_a001, moving_image_a002, result_image_a002_rigid, result_image_a002_affine
del coverslip_rotated_wf_images
gc.collect()

### Deskew all raw OPSIM images

In [ ]:
# deskew one channel/angle/phase to get the size of the image
test_deskew = deskew(sim_images[0,0,0,:],tilt_angle,ds,pixel_size)
n_coverslip_z = test_deskew.shape[0]
n_coverslip_y = test_deskew.shape[1]
n_coverslip_x = test_deskew.shape[2]

# create array to hold all deskewed images
coverslip_opsim_images = np.zeros((n_colors,n_angles,n_phases,n_coverslip_z,n_coverslip_y,n_coverslip_x),dtype=np.uint16)

# perform deskewing for all raw opsim views
for channel_idx in range(n_colors):
    for angle_idx in range(n_angles):
        for phase_idx in range(n_phases):
            coverslip_opsim_images[channel_idx,angle_idx,phase_idx,:] = deskew(np.flipud(sim_images[channel_idx,angle_idx,phase_idx,:,:,:]),tilt_angle,ds,pixel_size)

# clean up variables
del test_deskew, sim_images
gc.collect()

### Rotate deskewed raw OPSIM images

In [ ]:
# pad images to be square to hold rotated images
max_size = np.maximum(n_coverslip_y,n_coverslip_x)

if max_size > n_coverslip_y:
    pad_amount_y = np.abs(max_size-n_coverslip_y)//2
else:
    pad_amount_y = 0
if max_size > n_coverslip_x:
    pad_amount_x = np.abs(max_size-n_coverslip_x)//2
else:
    pad_amount_x = 0

pad_width = ((0,0),(0,0),(0,0),(0,0),(pad_amount_y,pad_amount_y+1),(pad_amount_x,pad_amount_x))
rotated_coverslip_opsim_images = np.pad(coverslip_opsim_images,pad_width=pad_width)

# rotation angle guesses from instrument design
rotation = [60,0,-60]

# perform rotation across all z planes for each channel + angle
for channel_idx in range(n_colors):
    for angle_idx in range(n_angles):
        for phase_idx in range(n_phases):
            for z_idx in range(n_coverslip_z):
                rotated_plane = rotate(rotated_coverslip_opsim_images[channel_idx,angle_idx,phase_idx,z_idx,:,:],
                                        angle=rotation[angle_idx],
                                        center=None,
                                        resize=False,
                                        mode='constant',
                                        cval=0,
                                        preserve_range=True)
                rotated_coverslip_opsim_images[channel_idx,angle_idx,phase_idx,z_idx,:,:] = rotated_plane

# clean up variables
del coverslip_opsim_images
gc.collect()

### Register deskewed raw OPSIM images using transformation matrices found using deskewed WF images

In [ ]:
# loop over all channel/angle/phase and apply transformations for angle_idx=[0,2] to angle_idx=1
for channel_idx in range(n_colors):
    for angle_idx in range(n_angles):
        for phase_idx in range(n_phases):
            if angle_idx == 0:
                result_image_transformix = itk.transformix_filter(rotated_coverslip_opsim_images[channel_idx,angle_idx,phase_idx,:,:,:].astype(np.float32),result_transform_parameters_a000_to_a001_rigid)
                result_image_transformix = itk.transformix_filter(result_image_transformix.astype(np.float32),result_transform_parameters_a000_to_a001_affine)
                rotated_coverslip_opsim_images[channel_idx,angle_idx,phase_idx,:,:,:]=result_image_transformix.astype(np.uint16)
            elif angle_idx == 2:
                result_image_transformix = itk.transformix_filter(rotated_coverslip_opsim_images[channel_idx,angle_idx,phase_idx,:,:,:].astype(np.float32),result_transform_parameters_a002_to_a001_rigid)
                result_image_transformix = itk.transformix_filter(result_image_transformix.astype(np.float32),result_transform_parameters_a002_to_a001_affine)
                rotated_coverslip_opsim_images[channel_idx,angle_idx,phase_idx,:,:,:]=result_image_transformix.astype(np.uint16)

### Visualize registered deskewed images in coverslip reference frame

In [ ]:
# add images to napari viewer with scale information
viewer = napari.view_image(rotated_coverslip_opsim_images[0,0,:],name='Registered deskewed angle 0',scale=(1,coverslip_dz,coverslip_dy,coverslip_dx))
viewer.add_image(rotated_coverslip_opsim_images[0,1,:],name='Registered deskewed angle 1',scale=(1,coverslip_dz,coverslip_dy,coverslip_dx))
viewer.add_image(rotated_coverslip_opsim_images[0,2,:],name='Registered deskewed angle 2',scale=(1,coverslip_dz,coverslip_dy,coverslip_dx))

# activate scale bar in physical units
viewer.scale_bar.unit = 'um'
viewer.scale_bar.visible = True

# label sliders in napari viewer
viewer.dims.axis_labels = ['c','a','p','z','y','x']

### Set ROI to middle of data and extract crop

In [ ]:
# set yx width of ROI
roi_sy=389
roi_sx=389
roi = rois.get_centered_roi([rotated_coverslip_opsim_images.shape[4]//2, rotated_coverslip_opsim_images.shape[5]//2], [roi_sy, roi_sx])

z_roi_low = 50
z_roi_high = 130

# extract crop
opsim_images_to_process = rotated_coverslip_opsim_images[:,:,:,z_roi_low:z_roi_high,roi[0]:roi[1],roi[2]:roi[3]]

# extract shape of cropped data
nz_cropped = opsim_images_to_process.shape[3]
ny_cropped = opsim_images_to_process.shape[4]
nx_cropped = opsim_images_to_process.shape[5]

### Visualize cropped data to pick "best" slice

In [ ]:
# add images to napari viewer with scale information
viewer = napari.view_image(opsim_images_to_process[0,0,:],name='Registered deskewed angle 0',scale=(1,coverslip_dz,coverslip_dy,coverslip_dx))
viewer.add_image(opsim_images_to_process[0,1,:],name='Registered deskewed angle 1',scale=(1,coverslip_dz,coverslip_dy,coverslip_dx))
viewer.add_image(opsim_images_to_process[0,2,:],name='Registered deskewed angle 2',scale=(1,coverslip_dz,coverslip_dy,coverslip_dx))

# activate scale bar in physical units
viewer.scale_bar.unit = 'um'
viewer.scale_bar.visible = True

# label sliders in napari viewer
viewer.dims.axis_labels = ['c','a','p','z','y','x']

### Find SIM peaks for each angle
See [this section of the I2K mcSIM tutorial](https://youtu.be/mDar-MjMtW0?t=6421) video and our [Nikon N-SIM processing tutorial](https://github.com/QI2lab/I2K2022-SIM/blob/0660281c28a809fdc09c36b00903b9230feb4d9c/example_004_Nikon_NSIM.ipynb) for how use the code block to find SIM frequencies.

In [ ]:
%matplotlib widget

# set "best" slice as determined from data
z_center_idx=15

for ch_idx in range(n_colors):

    # extract iamges
    image_set = opsim_images_to_process[ch_idx,:,:,z_center_idx,:,:]

    # calculate FFTs on grid with correct spatial dimensions
    dx = np.round(coverslip_dx,3)
    dy = np.round(coverslip_dy,3)
    nx = nx_cropped
    ny = ny_cropped
    fxs = analysis_tools.get_fft_frqs(nx, dx)
    df = fxs[1] - fxs[0]
    fys = analysis_tools.get_fft_frqs(ny, dy)
    ff = np.sqrt(np.expand_dims(fxs, axis=0)**2 + np.expand_dims(fys, axis=1)**2)
    for ii in range(image_set.shape[0]):
        ft = fft.fftshift(fft.fft2(fft.ifftshift(np.squeeze(image_set[ii, 0, :]))))

        figh = plt.figure()
        plt.title('Channel='+str(ch_idx)+", Angle="+str(ii))
        plt.imshow(np.abs(ft), norm=PowerNorm(gamma=0.1,vmin=65,vmax=1.5e7),
                    extent=[fxs[0] - 0.5 * df, fxs[-1] + 0.5 * df, fys[-1] + 0.5 * df, fys[0] - 0.5 * df])
        plt.show()

### Setup 2D-SIM processing

In [ ]:
# frequency guess from above. in k_x, k_y order
frq_guess = [[-1.28,2.26], [-2.6,0.01], [-1.27,-2.25]]
# phase guesses from prior knowledge
phase_guess = [[0, -4*np.pi/3,-2*np.pi/3], [0, -4*np.pi/3,-2*np.pi/3], [0, -4*np.pi/3,-2*np.pi/3]]

# algorithm parameters
wiener_parameter = 0.25
band0_exclusion_fraction = 0.5

### Load point spread function and calculate optical transfer function for each angle

2D PSF extracted from OPSIM data of beads by Reto Fiolka and Peter Brown using [qi2lab localize-psf package](https://www.github.com/qi2lab/localize-psf).  

In [ ]:
# load PSF from disk and tranpose to match metadata from raw data
psf2d = np.transpose(tifffile.imread(psf_file_path))
ny_psf, nx_psf = psf2d.shape

# create one PSF for each angle
psfs = np.asarray([rotate(psf2d, rotation[0], center=None, resize=False, mode='constant', cval=0, preserve_range=True),
                   psf2d,
                   rotate(psf2d, rotation[2], center=None, resize=False, mode='constant', cval=0, preserve_range=True)])

# pad to get size of image
if np.mod(ny_psf + ny_cropped, 2) != 0 or np.mod(nx_psf + nx_cropped, 2) != 0:
    raise ValueError()

# pad each PSF
ny_pad = int((ny_cropped - ny_psf) / 2)
nx_pad = int((nx_cropped - nx_psf) / 2)
psfs_padded = []
for p in psfs:
    psfs_padded.append(np.pad(p, ((ny_pad, ny_pad), (nx_pad, nx_pad)), mode="constant"))
psfs_padded = np.asarray(psfs_padded)

# get spatial frequencies on correct grid
fxs = fft.fftshift(fft.fftfreq(nx_cropped, coverslip_dx))
df = fxs[1] - fxs[0]
fys = fft.fftshift(fft.fftfreq(ny_cropped, coverslip_dy))
ff = np.sqrt(np.expand_dims(fxs, axis=0)**2 + np.expand_dims(fys, axis=1)**2)

# create OTF for each angle
fmax = 1 / (0.5 * emission_wavelengths / na)
otfs = []
for p in psfs_padded:
    otf = fft.fftshift(fft.fft2(fft.ifftshift(p)))
    otf = otf / np.max(np.abs(otf))
    otf[np.abs(otf) < 0.005] = 0
    otf[ff > fmax] = 0
    otf = np.abs(otf)

    otfs.append(otf)
otfs = np.asarray(otfs)

### Run 2D-SIM processing for z slice used to calculate frequencies above and extract reconstruction parameters
  
Modifications to [qi2lab mcSIM package](https://www.github.com/qi2lab/mcSIM) to handle OPSIM data by Peter Brown.

In [ ]:
# define save path
save_path = Path("data", "example_005", "reconstruction", "center_z_plane")

# create SIM reconstruction object for z_center_idx 
dset_one = sim.SimImageSet({'pixel_size': coverslip_dx, 'na': na, 'wavelength': emission_wavelengths},
                               opsim_images_to_process[0, :, :, z_center_idx, :],
                               frq_guess=frq_guess,
                               phases_guess=phase_guess,
                               wiener_parameter=wiener_parameter,
                               phase_estimation_mode="wicker-iterative",
                               max_phase_err=40 * np.pi / 180,
                               combine_bands_mode="fairSIM", 
                               otf=otfs,
                               fmax_exclude_band0=band0_exclusion_fraction,
                               min_p2nr=0.8,
                               gain=1, 
                               background=0,
                               save_dir=save_path,
                               save_suffix='_z'+str(z_center_idx).zfill(3)+'_ch'+str(0).zfill(3),
                               interactive_plotting=False)

# perform reconstruction, plot figures, save
dset_one.reconstruct()
dset_one.plot_figs()
dset_one.save_imgs()
dset_one.save_result()

# save reconstruction parameter for use on full stack
frqs = np.array(dset_one.frqs, copy=True)
phases = np.array(dset_one.phases, copy=True)
mod_depths = np.array(dset_one.mod_depths, copy=True)

# clean up variables
del dset_one
gc.collect()

### Reconstruct full z stack using parameters from selected z plane

In [ ]:
# define save path
save_path = Path("data", "example_005", "reconstruction", "full_z_stack")

# loop over channels and z slices, perform 2D at each slice using parameters from above reconstruction
for ch_idx in range(n_colors):
    for z_idx in range(nz_cropped):
        imgs_recon = opsim_images_to_process[ch_idx, :, :, z_idx,:,:]

        imgset = sim.SimImageSet({'pixel_size': coverslip_dx, 'na': na, 'wavelength': emission_wavelengths}, 
                                imgs_recon,
                                frq_guess=frqs,
                                frq_estimation_mode="fixed",
                                phases_guess=phases,
                                phase_estimation_mode="fixed",
                                max_phase_err=40 * np.pi / 180,
                                mod_depths_guess=mod_depths,
                                use_fixed_mod_depths=True,
                                wiener_parameter=wiener_parameter,
                                combine_bands_mode="fairSIM",
                                fmax_exclude_band0=band0_exclusion_fraction,
                                otf=otfs,
                                gain=1, 
                                background=0,
                                save_dir=save_path, 
                                save_suffix='_z'+str(z_idx).zfill(3)+'_ch'+str(0).zfill(3),
                                interactive_plotting=False)
        imgset.reconstruct()
        imgset.save_result()
        imgset.save_imgs()
        imgset.plot_figs()

        # create variables to hold widefield and SIM SR images
        if ch_idx == 0 and z_idx == 0:
            wf_images = np.zeros((n_colors,nz_cropped,imgset.widefield.shape[0],imgset.widefield.shape[1]),dtype=np.float32)
            SR_images = np.zeros((n_colors,nz_cropped,imgset.sim_sr.shape[0],imgset.sim_sr.shape[1]),dtype=np.float32)

        # store widefield and SIM SR images for display
        wf_images[ch_idx,z_idx,:]=imgset.widefield
        SR_images[ch_idx,z_idx,:]=imgset.sim_sr

        # clean up variables
        del imgs_recon, imgset
        gc.collect()

### Display results

In [ ]:
# define colormaps
colormaps = ['bop purple', 'bop blue', 'bop orange']

# add images to napari viewer with scale information, colormaps, and additive blending
for ch_idx in range(n_colors):
    if ch_idx == 0:
        viewer = napari.view_image(wf_images[ch_idx,:],name='Widefield CH '+str(ch_idx).zfill(2),scale=(coverslip_dz,coverslip_dy,coverslip_dx),colormap = colormaps[ch_idx],blending='additive',contrast_limits=[0,2**16-1])
        viewer.add_image(SR_images[ch_idx,:],name='SR CH '+str(ch_idx).zfill(2),scale=(coverslip_dz,coverslip_dy/2,coverslip_dx/2),colormap = colormaps[ch_idx],blending='additive',contrast_limits=[0,2**16-1])
    else:
        viewer.add_image(wf_images[ch_idx,:],name='Widefield CH '+str(ch_idx).zfill(2),scale=(coverslip_dz,coverslip_dy,coverslip_dx),colormap = colormaps[ch_idx],blending='additive',contrast_limits=[0,2**16-1])
        viewer.add_image(SR_images[ch_idx,:],name='SR CH '+str(ch_idx).zfill(2),scale=(coverslip_dz,coverslip_dy/2,coverslip_dx/2),colormap = colormaps[ch_idx],blending='additive',contrast_limits=[0,2**16-1])

# activate scale bar in physical units
viewer.scale_bar.unit = 'um'
viewer.scale_bar.visible = True

### Write widefield and OPSIM reconstructions to disk as OME-TIFF

In [ ]:
# widefield
save_path_wf = Path("data", "example_005", "reconstruction", "reconstruction_wf.ome.tif")
wf_metadata = { 'axes': 'CZYX',
                'Pixels': {'PhysicalSizeX': coverslip_dx,
                           'PhysicalSizeXUnit': 'µm',
                           'PhysicalSizeY': coverslip_dy,
                           'PhysicalSizeYUnit': 'µm',
                           'PhysicalSizeZ': coverslip_dz,
                           'PhysicalSizeZUnit': 'µm'}
              }
tifffile.imwrite(save_path_wf,wf_images,metadata=wf_metadata)

# opsim
save_path_opsim = Path("data", "example_005", "reconstruction", "reconstruction_opsim.ome.tif")
opsim_metadata = { 'axes': 'CZYX',
                'Pixels': {'PhysicalSizeX': coverslip_dx//2,
                           'PhysicalSizeXUnit': 'µm',
                           'PhysicalSizeY': coverslip_dy//2,
                           'PhysicalSizeYUnit': 'µm',
                           'PhysicalSizeZ': coverslip_dz,
                           'PhysicalSizeZUnit': 'µm'}
              }
tifffile.imwrite(save_path_opsim,SR_images,metadata=opsim_metadata)